In [1]:
import pandas as pd

In [2]:
citizen_df = pd.read_excel('../../excel-files/citizen.xlsx')
service_df = pd.read_excel('../../excel-files/emon.service.xlsx')
request_df_sheets = pd.read_excel('../../excel-files/request.xlsx', sheet_name=None)

In [3]:
request_df = pd.concat(request_df_sheets.values())

In [4]:
request_df.rename(columns={"createc_date": "created_date", "service_id": "serviceid"}, inplace=True)
service_df.rename(columns={"_id": "serviceid"}, inplace=True)

In [5]:
request_df["created_date"] = pd.to_datetime(request_df["created_date"])

In [6]:
request_df['month'] = request_df['created_date'].dt.month
service_month_counts = request_df.groupby(['serviceid', 'month']).size().reset_index(name='count')
most_requested = service_month_counts.loc[
    service_month_counts.groupby('serviceid')['count'].idxmax()
]

In [7]:
all_users = set(citizen_df['userid']).union(set(request_df['userid']))
all_services = set(service_df['serviceid']).union(set(request_df['serviceid']))

In [8]:
len(all_users), len(all_services)

(50000, 2785)

In [9]:
user_map = {user_id: idx for idx, user_id in enumerate(sorted(all_users))}
service_map = {service_id: idx for idx, service_id in enumerate(sorted(all_services))}

In [10]:
with open('user_list.txt', 'w') as f:
    for user, idx in user_map.items():
        f.write(f"{user} {idx}\n")

with open('item_list.txt', 'w') as f:
    for service, idx in service_map.items():
        f.write(f"{service} {idx}\n")

In [11]:
request_df['userid'] = request_df['userid'].map(user_map)
request_df['serviceid'] = request_df['serviceid'].map(service_map)

In [12]:
request_df = request_df.sort_values(by='created_date')

In [13]:
split_idx = int(len(request_df) * 0.8)
train_df = request_df.iloc[:split_idx]
test_df = request_df.iloc[split_idx:]

In [14]:
missing_users = set(test_df['userid']) - set(train_df['userid'])
if missing_users:
    extra_train_data = test_df[test_df['userid'].isin(missing_users)]
    train_df = pd.concat([train_df, extra_train_data])
    test_df = test_df[~test_df['userid'].isin(missing_users)]

In [15]:
def save_format(filename, data):
    grouped = data.groupby('userid')['serviceid'].apply(lambda x: ' '.join(map(str, x)))
    with open(filename, 'w') as f:
        for user, items in grouped.items():
            f.write(f"{user} {items}\n")  
save_format('train.txt', train_df)
save_format('test.txt', test_df)

In [16]:
num_users = len(user_map)
num_items = len(service_map)

In [17]:
all_agency = set(service_df['govAgencyId'])
all_service_types = set(service_df["serviceType"])
all_months = set({"month1", "month2", "month3", "month4", "month5", "month6", "month7", "month8", "month9", "month10", "month11", "month12"})
all_age_group = set(["unknown", "under_27", "27_34", "35_41", "42_50", "51_60", "over_60"])
len(all_agency), len(all_service_types), len(all_months), len(all_age_group)

(442, 8, 12, 7)

In [18]:
kg_entity_map = {agency_id: idx for idx, agency_id in enumerate(sorted(all_agency))}
kg_entity_map.update({service_type: idx + len(all_agency) for idx, service_type in enumerate(sorted(all_service_types))})
kg_entity_map.update({month: idx + len(all_agency) + len(all_service_types) for idx, month in enumerate(sorted(all_months))})
kg_entity_map.update({age_group: idx + len(all_agency) + len(all_service_types) + len(all_months) for idx, age_group in enumerate(sorted(all_age_group))})

In [19]:
entity2id = {}
    
for service, idx in service_map.items():
    entity2id[service] = idx
    
user_offset = len(service_map)
for user, idx in user_map.items():
    entity2id[user] = idx + user_offset
    
offset = len(service_map) + len(user_map)
for other_entity, idx in kg_entity_map.items():
    entity2id[other_entity] = idx + offset


In [20]:
service_df = service_df.merge(most_requested, on='serviceid', how='left')


In [21]:
def assign_age_group(age):
    if pd.isna(age) or not isinstance(age, (int, float)):
        return "unknown"
    if age < 27:
        return "under_27"
    elif age < 35:
        return "27_34"
    elif age < 42:
        return "35_41"
    elif age < 51:
        return "42_50"
    elif age < 61:
        return "51_60"
    else:
        return "over_60"
citizen_df['age_group'] = citizen_df['age'].apply(assign_age_group)

relation2id = {
    "provided_by": 0,
    "type_of": 1,
    "most_requested": 2,
    "belongs_to": 3,
}

kg_triples = []

for _, row in service_df.iterrows():
    service_id = entity2id[row['serviceid']]
    agency_id = entity2id[row['govAgencyId']]
    service_type_id = entity2id[row['serviceType']]

    kg_triples.append(f"{service_id} {relation2id['provided_by']} {agency_id}")
    kg_triples.append(f"{service_id} {relation2id['type_of']} {service_type_id}")
    
    if pd.notna(row['month']):
        month_id = entity2id['month' + str(int(row['month']))]
        kg_triples.append(f"{service_id} {relation2id['most_requested']} {month_id}")
        
for _, row in citizen_df.iterrows():
    user_id = entity2id[row['userid']]
    age_group_id = entity2id[row['age_group']]
    kg_triples.append(f"{user_id} {relation2id['belongs_to']} {age_group_id}")

In [22]:
with open("kg.txt", "w") as f:
    for triple in kg_triples:
        f.write(triple + "\n")

with open("entity_list.txt", "w") as f:
    for ent, eid in entity2id.items():
        f.write(f"{ent} {eid}\n")

with open("relation_list.txt", "w") as f:
    for rel, rid in relation2id.items():
        f.write(f"{rel} {rid}\n")
